<a href="https://colab.research.google.com/github/anirbansen3027/SecureAndPrivateAI/blob/master/6_Remote_Tensors_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#installing syft on the machine
!pip install syft

In [0]:
#importing necessary libraries
import torch as th
import syft as sy

In [0]:
#
hook = sy.TorchHook(th)

W0713 14:00:19.743882 139849308526464 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
#Creating the vrtual workers
bob = sy.VirtualWorker(hook, id = "bob")

In [0]:
#Looking at the objects bob has
bob._objects

{}

In [0]:
#creating a tensor to send
x = th.tensor([1,2,3,4])

In [0]:
#sending the object to bob
x = x.send(bob)

In [0]:
#Looking at the objects bob has
bob._objects

{48574571104: tensor([1, 2, 3, 4])}

In [0]:
#X is a pointer pointing to bob
x.location

<VirtualWorker id:bob #objects:1>

In [0]:
#Cheking if location that x is pointing to and bob is same
x.location == bob

True

In [0]:
#id of x
x.id

26371767005

In [0]:
#id of the worker it is pointing to
x.id_at_location

48574571104

In [0]:
#Owner worker of x -- me which is created without our knowledge
x.owner

<VirtualWorker id:me #objects:0>

In [0]:
#The local worker of the hook
hook.local_worker

<VirtualWorker id:me #objects:0>

In [0]:
#Getting the data
x = x.get(bob)
x

tensor([1, 2, 3, 4])

In [0]:
bob._objects

{}

In [0]:
alice = sy.VirtualWorker(hook, id = "alice")

In [0]:
x = th.tensor([1,2,3,4])
x_ptr = x.send(bob,alice)

In [0]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:44787368119 -> bob:6419262976]
	-> (Wrapper)>[PointerTensor | me:63168799769 -> alice:92813206819]

In [0]:
x_ptr.child.child

{'alice': (Wrapper)>[PointerTensor | me:63168799769 -> alice:92813206819],
 'bob': (Wrapper)>[PointerTensor | me:44787368119 -> bob:6419262976]}

In [0]:
x = x_ptr.get()
x

[tensor([1, 2, 3, 4]), tensor([1, 2, 3, 4])]

In [0]:
x_ptr = th.tensor([1,2,3,4,5]).send(bob,alice)
x_ptr.get(sum_results=True)

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
z = x+y
z.get()

tensor([2, 3, 4, 5, 6])

In [0]:
bob._objects

{41210247635: tensor([1, 1, 1, 1, 1]), 59151849333: tensor([1, 2, 3, 4, 5])}

In [0]:
z = th.add(x,y)
z.get()

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5],requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1],requires_grad=True).send(bob)

In [0]:
z = (x+y).sum()
z.backward()
x = x.get()

In [0]:
x.grad

tensor([1., 1., 1., 1., 1.])

In [0]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
input = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad=True).send(bob)
target = th.tensor([[1.],[0],[1],[1]],requires_grad=True).send(bob)
weight = th.tensor([[0.],[0.]],requires_grad=True).send(bob)

In [0]:
for _ in range(10):
  pred = input.mm(weight)
  loss = ((pred - target)**2).sum()
  loss.backward()
  weight.data.sub_(weight.grad*0.1)
  weight.grad*=0
  print(loss.get().data)

tensor(3.)
tensor(1.5600)
tensor(1.2432)
tensor(1.1372)
tensor(1.0849)
tensor(1.0538)
tensor(1.0344)
tensor(1.0220)
tensor(1.0141)
tensor(1.0090)


In [0]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4]).send(bob)
bob._objects

{98236608274: tensor([1, 2, 3, 4])}

In [0]:
x.child.garbage_collect_data

True

In [0]:
del x

In [0]:
bob._objects

{}

In [0]:
for _ in range(1000):
  x = th.tensor([1,2,3,4]).send(bob)
bob._objects

{55221387730: tensor([1, 2, 3, 4])}

In [0]:
x = th.tensor([1,2,3,4]).send(bob)
y = th.tensor([1,1,1,1])
z = x+y

TensorsNotCollocatedException: ignored

In [0]:
bob.clear_objects()
alice.clear_objects()
x = th.tensor([1,2,3,4,5]).send(bob)
x = x.send(alice)

In [0]:
bob._objects

{16714972279: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{53565952802: (Wrapper)>[PointerTensor | alice:53565952802 -> bob:16714972279]}

In [0]:
x

(Wrapper)>[PointerTensor | me:78987048329 -> alice:53565952802]

In [0]:
y=x+x

In [0]:
bob._objects

{16714972279: tensor([1, 2, 3, 4, 5]),
 36537578454: tensor([ 2,  4,  6,  8, 10])}

In [0]:
alice._objects

{13328586251: (Wrapper)>[PointerTensor | alice:13328586251 -> bob:36537578454],
 53565952802: (Wrapper)>[PointerTensor | alice:53565952802 -> bob:16714972279]}

In [0]:
john = sy.VirtualWorker(hook,id = "john")
bob.clear_objects()
alice.clear_objects()
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(john)

In [0]:
bob._objects

{77361687237: tensor([1, 2, 3, 4, 5]), 90771514427: tensor([1, 2, 3, 4, 5])}

In [0]:
z = x+y

TensorsNotCollocatedException: ignored

In [0]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:88566755795 -> bob:77361687237]

In [0]:
alice._objects

{}

In [0]:
bob._objects

{77361687237: tensor([1, 2, 3, 4, 5]), 90771514427: tensor([1, 2, 3, 4, 5])}

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{90771514427: tensor([1, 2, 3, 4, 5])}

In [0]:
bob.clear_objects()
alice.clear_objects()
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

{22832302184: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{81198503146: (Wrapper)>[PointerTensor | alice:81198503146 -> bob:22832302184]}

In [0]:
del x

In [0]:
bob._objects,alice._objects # both get garbage collected

({}, {})